In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import scipy
import random
import warnings
warnings.filterwarnings('ignore')

penalty_list = ['none','l1','l2','elasticnet']
# penalty_list = ['elasticnet']

In [2]:
def njobs(x):
    if x == -1:
        return 32
    else:
        return x
def toOneHot(x):
    result = [];
    for i in x:
        if i =='none':
            result.append([1,0,0,0])
        elif i=='l1':
            result.append([0,1,0,0])
        elif i=='l2':
            result.append([0,0,1,0])
        else:
            result.append([0,0,0,1])
    return result

def models_fit(models, X, y):
    for k, v in X.items():
        models[k].fit(v.values, y.loc[v.index].values)

def models_pred(models, X):
    pred = pd.DataFrame()
    for k, v in X.items():
        temp = models[k].predict(v.values)
        for i in range(len(temp)):
            if temp[i] < 0:
                temp[i] = 0
        pred = pred.append(pd.DataFrame(temp, index = v.index, columns = ['time'] ))
    return pred.sort_index()

# def feature_split(features)

In [3]:
def preprocess(old_X, old_scaler = None, traing = True):
    X = pd.DataFrame(old_X.values, columns = old_X.columns)   
    X[:] = old_X.values
    scaler = old_scaler
    X['n_jobs'] = X.loc[:,'n_jobs'].apply(lambda x: 32 if x == -1 else x)
#     X['alpha'] = X['alpha'].apply(lambda x: -np.log10(x) )    
        
    X['new1'] = X['max_iter'] * X['n_samples']*X['n_features']*X['n_classes']/X['n_jobs']
    X['new2'] = X['n_classes'] * X['n_informative']
    X['new3'] = X['n_clusters_per_class'] * X['n_informative']
#     X['new2'] = X['max_iter'] * X['n_samples']*X['n_features']/X['n_jobs']/X['n_clusters_per_class']    
    penalty = X['penalty']
    n_jobs = X['n_jobs']
    
    X = X.drop(['penalty'], axis = 1)
    X = X.drop(['random_state'], axis = 1)
    X = X.drop(['scale'], axis = 1)    
#     X = X.drop(['n_jobs'], axis = 1)
#     X = X.drop(['max_iter'], axis = 1)
#     X = X.drop(['n_samples'], axis = 1)
#     X = X.drop(['n_features'], axis = 1)
#     X = X.drop(['n_clusters_per_class'], axis = 1)

    
    if traing and (scaler==None):
        scaler = preprocessing.StandardScaler()
        X[:] = scaler.fit_transform(X)
    else:
        X[:] = scaler.transform(X)
        
#     X['n_jobs'] = n_jobs
    split_features  = {}
    for i in penalty_list:
        split_features[i] =  X.loc[penalty == i]
    
#     X[['none', 'l1','l2','elasticnet']] = pd.DataFrame(toOneHot(penalty))
    return split_features, scaler

In [4]:
#load data
data = pd.read_csv("train.csv", index_col='id')

In [5]:
#valid set test
# def all_pred(models, features):
hidden_layer = (300, 300)
max_iter=300000

valid_iter = 50

models = {}
score = 0
for i in range(valid_iter):
    
    data = data.sample(frac=1).reset_index(drop=True)

    X_data = data.drop(['time'], axis = 1)
    y_data = data[['time']]

    features = X_data.loc[  :300 ].sort_index()
    labels = y_data.loc[ :300 ].sort_index()

    valid_features = X_data.loc[  300: ].sort_index()
    valid_labels = y_data.loc[ 300: ].sort_index()
    
    processed_features, fitted_scaler = preprocess(features)
    
    for k,v in processed_features.items():
        models[k] = MLPRegressor(hidden_layer_sizes=hidden_layer,max_iter=max_iter)
    models_fit(models, processed_features, labels)
    
    processed_valid,_ = preprocess(valid_features, fitted_scaler,False)
    # processed_valid
    score += metrics.mean_squared_error( models_pred(models, processed_valid).values, valid_labels.values)
print (score/valid_iter)

0.992447704624032


In [6]:
processed_X,_ = preprocess(X_data, fitted_scaler, False)
# cross_val_score(model, cv=5)
models_fit(models, processed_X, y_data)

In [7]:
test_data = pd.read_csv("test.csv", index_col='id')

In [8]:
processed_test_data, _ = preprocess(test_data, fitted_scaler, False)
# processed_test_data.head()

In [9]:
test_results = models_pred(models, processed_test_data)

In [14]:
#output the results
test_results.to_csv('output.csv',index_label = ['Id'])